# Entrenamiento

🛑 Recuerde cambiar la dirección por la ubicación de la carpeta Workshop

---


In [ ]:
folder = ''

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import pylab as pl

In [ ]:
# Cargar el modelo YOLO preentrenado
model = YOLO('yolov8n.pt')

# Cargar una imagen
img = f'{folder}/stop.jpg'

# Realizar la detección
results = model(img)

img_result = cv2.resize(results[0].plot(), (900, 680))

# Mostrar los resultados
cv2_imshow(img_result)

# Entrenamiento

In [ ]:
from ultralytics import YOLO
import seaborn as sns
import pylab as pl
import random
import glob
import os
import numpy as np

In [ ]:
!unzip -q -n {folder}/datos.zip -d {folder}/

In [ ]:
images = glob.glob(f'{folder}/datos/*.jpg')
np.random.shuffle(images)

In [ ]:
train_images = images[:int(len(images)*.70)]
test_images = images[int(len(images)*.70):int(len(images)*.90)]
valid_images = images[int(len(images)*.90):]

In [ ]:
for img in train_images:
  train_img_folder = f'{folder}/datos/train/images/'
  os.makedirs(train_img_folder, exist_ok=True)
  command = f'cp {img} {train_img_folder}'
  os.system(command)

  train_lbl_folder = f'{folder}/datos/train/labels/'
  os.makedirs(train_lbl_folder, exist_ok=True)
  lbl = img.replace('.jpg','.txt')
  command = f'cp {lbl} {train_lbl_folder}'
  os.system(command)

for img in valid_images:
  valid_img_folder = f'{folder}/datos/valid/images/'
  os.makedirs(valid_img_folder, exist_ok=True)
  command = f'cp {img} {valid_img_folder}'
  os.system(command)

  valid_lbl_folder = f'{folder}/datos/valid/labels/'
  os.makedirs(valid_lbl_folder, exist_ok=True)
  lbl = img.replace('.jpg','.txt')
  command = f'cp {lbl} {valid_lbl_folder}'
  os.system(command)

for img in test_images:
  test_img_folder = f'{folder}/datos/test/images/'
  os.makedirs(test_img_folder, exist_ok=True)
  command = f'cp {img} {test_img_folder}'
  os.system(command)

  test_lbl_folder = f'{folder}/datos/test/labels/'
  os.makedirs(test_lbl_folder, exist_ok=True)
  lbl = img.replace('.jpg','.txt')
  command = f'cp {lbl} {test_lbl_folder}'
  os.system(command)

In [ ]:
print(f'{folder}/datos/train/images')
print(f'{folder}/datos/valid/images')
print(f'{folder}/datos/test/images')

In [ ]:
# Cargando el modelo
model = YOLO('yolov8x.pt')

# Entrenando el Modelo
model.train(data = f'{folder}/data.yaml',
            epochs = 3,
            seed = 42,
            batch = 8,
            workers = 4,
            pretrained = True)

last_train = np.sort(glob.glob('/content/runs/detect/train3'))[-1]

best_model = f'{last_train}/weights/best.pt'
command = f'cp {best_model} {folder}/'
os.system(command)

last_model = f'{last_train}/weights/last.pt'
command = f'cp {last_model} {folder}/'
os.system(command)

In [ ]:
# Cargando el mejor modelo
model = YOLO(f'{folder}/best.pt')

# Evaluando el modelo sobre los datos de prueba
metrics = model.val(conf = 0.25, split = 'test', data=f'{folder}/data.yaml')

In [ ]:
# Barplots
ax = sns.barplot(x=['mAP50-95', 'mAP50', 'mAP75'], y=[metrics.box.map, metrics.box.map50, metrics.box.map75])

ax.set_title('YOLO Evaluation Metrics')
ax.set_xlabel('Métrica')
ax.set_ylabel('Valor')

fig = pl.gcf()
fig.set_size_inches(8, 6)

for p in ax.patches:
    ax.annotate('{:.3f}'.format(p.get_height()), (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')

pl.show()

# Probando resultados

In [ ]:
def pet_detect(img_path):
    # Carga la Imagen
    img = cv2.imread(img_path)

    # Aplica el modelo de detección
    detect_result = model(img)

    # Marca la detección
    detect_img = detect_result[0].plot()

    # Convierte la Imagen a RGB
    detect_img = cv2.cvtColor(detect_img, cv2.COLOR_BGR2RGB)

    return detect_img

In [ ]:
custom_image_dir = f'{folder}/datos/test/images'

# Crea una lista de imágenes
image_files = os.listdir(custom_image_dir)

# Selecciona 16 imágenes aleatorias
selected_images = random.sample(image_files, 16)

# Crea una grilla para dibujar los resultados
fig, axes = pl.subplots(nrows=4, ncols=4, figsize=(15, 15))

# Itera sobre las imágenes para la detección
for i, img_file in enumerate(selected_images):

    row_idx = i // 4
    col_idx = i % 4

    # Detecta por cada imagen
    img_path = os.path.join(custom_image_dir, img_file)
    detect_img = pet_detect(img_path)

    # Dibuja el resultado
    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

pl.subplots_adjust(wspace=0.05, hspace=0.05)